# **Diya**

In [1]:
import pandas as pd
import joblib

In [6]:
df=pd.read_excel("Constraint_English_Train.xlsx")

In [8]:
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [10]:
df.shape

(6420, 3)

In [12]:
df1=pd.read_excel("english_test_with_labels.xlsx")

In [14]:
df1.shape

(2140, 3)

In [16]:
df2=pd.read_excel("Constraint_English_Val.xlsx")

In [18]:
df2.shape

(2140, 3)

In [20]:
df_combined=pd.concat([df,df1,df2])

In [22]:
df_combined.shape

(10700, 3)

In [24]:
6420+2140+2140

10700

In [26]:
df_combined.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [28]:
df_combined["label"].value_counts()

label
real    5600
fake    5100
Name: count, dtype: int64

In [30]:
df_combined["label"]=df_combined["label"].apply(lambda x:1 if x=="real" else 0)

In [32]:
df_combined['label'].value_counts()

label
1    5600
0    5100
Name: count, dtype: int64

In [34]:
nan_values = df_combined.isnull()
print(df_combined[nan_values.any(axis=1)])

Empty DataFrame
Columns: [id, tweet, label]
Index: []


In [36]:
df_combined = df_combined.dropna()

In [38]:
df_combined['label'].value_counts()

label
1    5600
0    5100
Name: count, dtype: int64

In [40]:
df_combined.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,1
1,2,States reported 1121 deaths a small rise from ...,1
2,3,Politically Correct Woman (Almost) Uses Pandem...,0
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1
4,5,Populous states can generate large case counts...,1


In [42]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_combined['tweet'],df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label'])

In [44]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

# Load pre-trained GloVe vectors
def load_glove_model(glove_path):
    glove_dict = {}
    with open(glove_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_dict[word] = vector
    return glove_dict

# Convert text data into GloVe-based sentence embeddings
def get_sentence_embedding(sentence, glove_dict, embedding_dim=100):
    words = word_tokenize(sentence.lower())
    word_vectors = [glove_dict[word] for word in words if word in glove_dict]
    if len(word_vectors) == 0:
        return np.zeros(embedding_dim)
    return np.mean(word_vectors, axis=0)

# Load GloVe vectors
glove_path = "C:/Users/91935/Downloads/glove.6B/glove.6B.50d.txt"  # Update this path

glove_dict = load_glove_model(glove_path)

# Convert text data to embeddings
X_train_vectorized = np.array([get_sentence_embedding(text, glove_dict) for text in x_train])
X_test_vectorized = np.array([get_sentence_embedding(text, glove_dict) for text in x_test])

# Train RandomForest model
clf = Pipeline([
    ('rfc', RandomForestClassifier())
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclfrf = clf.score(X_test_vectorized, y_test)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


              precision    recall  f1-score   support

           0       0.87      0.90      0.89      1020
           1       0.91      0.88      0.89      1120

    accuracy                           0.89      2140
   macro avg       0.89      0.89      0.89      2140
weighted avg       0.89      0.89      0.89      2140



In [46]:
import joblib

# Save the trained model
joblib.dump(clf, 'pclfrf.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfrf.pkl')

C:\Users\91935\NLP\newpkls\pclfrf.pkl

In [48]:
from sklearn.svm import SVC
clf = Pipeline([
    ('svc', SVC(kernel='linear'))
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclfsvc = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfsvc.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfsvc.pkl')

              precision    recall  f1-score   support

           0       0.87      0.84      0.86      1020
           1       0.86      0.89      0.87      1120

    accuracy                           0.86      2140
   macro avg       0.86      0.86      0.86      2140
weighted avg       0.86      0.86      0.86      2140



C:\Users\91935\NLP\newpkls\pclfsvc.pkl

In [50]:
from sklearn.linear_model import LogisticRegression
clf = Pipeline([
    ('logreg', LogisticRegression())
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclflr = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclflr.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclflr.pkl')

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      1020
           1       0.86      0.89      0.87      1120

    accuracy                           0.86      2140
   macro avg       0.86      0.86      0.86      2140
weighted avg       0.86      0.86      0.86      2140



C:\Users\91935\NLP\newpkls\pclflr.pkl

In [52]:
from sklearn.neighbors import KNeighborsClassifier
clf = Pipeline([
    ('knn', KNeighborsClassifier(n_neighbors=5))
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclfdt = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfdt.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfdt.pkl')

              precision    recall  f1-score   support

           0       0.94      0.77      0.85      1020
           1       0.82      0.96      0.88      1120

    accuracy                           0.87      2140
   macro avg       0.88      0.86      0.87      2140
weighted avg       0.88      0.87      0.87      2140



C:\Users\91935\NLP\newpkls\pclfdt.pkl

In [54]:
from sklearn.tree import DecisionTreeClassifier
clf = Pipeline([
    ('dt', DecisionTreeClassifier())
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclfdt = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfdt.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfdt.pkl')


              precision    recall  f1-score   support

           0       0.78      0.77      0.77      1020
           1       0.79      0.80      0.79      1120

    accuracy                           0.78      2140
   macro avg       0.78      0.78      0.78      2140
weighted avg       0.78      0.78      0.78      2140



C:\Users\91935\NLP\newpkls\pclfdt.pkl

In [56]:
from sklearn.ensemble import GradientBoostingClassifier
clf = Pipeline([
    ('gb', GradientBoostingClassifier())
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclfgb = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfgb.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfgb.pkl')

              precision    recall  f1-score   support

           0       0.90      0.86      0.88      1020
           1       0.88      0.91      0.90      1120

    accuracy                           0.89      2140
   macro avg       0.89      0.89      0.89      2140
weighted avg       0.89      0.89      0.89      2140



C:\Users\91935\NLP\newpkls\pclfgb.pkl

In [58]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [60]:
def stemming_text(text):
    ps=PorterStemmer()
    stop_words=set(stopwords.words('english'))
    words=word_tokenize(text.lower())
    processed_words=[ps.stem(word) for word in words if word.isalnum() and word not in stop_words]

    return " ".join(processed_words)

In [62]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

def lemmatize_text(text):
    stop_words = set(stopwords.words("english"))
    lemmatizer = WordNetLemmatizer()

    tokens = word_tokenize(text)
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalnum() and token.lower() not in stop_words]

    return " ".join(filtered_tokens)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [64]:
df_combined['stemming_content']=df_combined['tweet'].apply(stemming_text)
df_combined['lemmatized_content']=df_combined['tweet'].apply(lemmatize_text)

In [66]:
df_combined.head()

,id,tweet,label,stemming_content,lemmatized_content
0,1,The CDC currently reports 99031 deaths. In gen...,1,cdc current report 99031 death gener discrep d...,CDC currently report 99031 death general discr...
1,2,States reported 1121 deaths a small rise from ...,1,state report 1121 death small rise last tuesda...,States reported 1121 death small rise last Tue...
2,3,Politically Correct Woman (Almost) Uses Pandem...,0,polit correct woman almost use pandem excus re...,Politically Correct Woman Almost Uses Pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,1,indiafightscorona 1524 covid test laboratori i...,IndiaFightsCorona 1524 COVID testing laborator...
4,5,Populous states can generate large case counts...,1,popul state gener larg case count look new cas...,Populous state generate large case count look ...


In [68]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_combined['stemming_content'],df_combined['label'], test_size=0.2, random_state=2022)

In [98]:
import numpy as np

def get_sentence_embedding(sentence, glove_dict, dim=300):
    words = sentence.split()
    embeddings = [glove_dict[word] for word in words if word in glove_dict]

    if not embeddings:
        return np.zeros(dim, dtype=np.float32)  # 🔥 Fix: Always return (300,)

    stacked = np.stack(embeddings)  # Convert list to array
    mean_embedding = np.mean(stacked, axis=0).astype(np.float32)

    return mean_embedding.reshape((dim,))  # 🔥 Fix: Always return shape (300,)

# ✅ Now, this will work!
X_train_vectorized = np.array([get_sentence_embedding(text, glove_dict, dim=300) for text in x_train])
X_test_vectorized = np.array([get_sentence_embedding(text, glove_dict, dim=300) for text in x_test])

print("Final X_train shape:", X_train_vectorized.shape)  # (8560, 300)
print("Final X_test shape:", X_test_vectorized.shape)    # (2140, 300)


ValueError: cannot reshape array of size 50 into shape (300,)

In [78]:
#import numpy as np

#def get_sentence_embedding(sentence, glove_dict, dim=300):
   # words = sentence.split()
    #embeddings = [glove_dict[word] for word in words if word in glove_dict]

    #if not embeddings:
        #return np.zeros(dim)  # Ensure a fixed-length vector if no words are found

    #return np.mean(embeddings, axis=0)  # Compute average embedding

# Convert text data into fixed-length vectors
#X_train_vectorized = np.array([get_sentence_embedding(text, glove_dict) for text in x_train])
#X_test_vectorized = np.array([get_sentence_embedding(text, glove_dict) for text in x_test])

#clf = Pipeline([
 #   ('rfc', RandomForestClassifier())
#])
#clf.fit(X_train_vectorized, y_train)

ValueError: setting an array element with a sequence.

In [152]:
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
pclfrfs = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfrfs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfrfs.pkl')

ValueError: setting an array element with a sequence.

In [36]:
from sklearn.svm import SVC
clf=Pipeline([
    ('svc',SVC())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfsvcs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfsvcs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfsvcs.pkl')

              precision    recall  f1-score   support

           0       0.91      0.81      0.86      1024
           1       0.84      0.93      0.88      1116

    accuracy                           0.87      2140
   macro avg       0.88      0.87      0.87      2140
weighted avg       0.88      0.87      0.87      2140



In [37]:
from sklearn.linear_model import LogisticRegression
clf=Pipeline([
    ('lr',LogisticRegression())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclflrs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclflrs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclflrs.pkl')

              precision    recall  f1-score   support

           0       0.85      0.78      0.81      1024
           1       0.81      0.88      0.84      1116

    accuracy                           0.83      2140
   macro avg       0.83      0.83      0.83      2140
weighted avg       0.83      0.83      0.83      2140



In [38]:
from sklearn.neighbors import KNeighborsClassifier
clf=Pipeline([
    ('knn',KNeighborsClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfknns=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfknns.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfknns.pkl')

              precision    recall  f1-score   support

           0       0.94      0.67      0.78      1024
           1       0.76      0.96      0.85      1116

    accuracy                           0.82      2140
   macro avg       0.85      0.81      0.81      2140
weighted avg       0.85      0.82      0.82      2140



In [39]:
from sklearn.tree import DecisionTreeClassifier
clf=Pipeline([
    ('dtc',DecisionTreeClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfdts=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfdts.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfdts.pkl')

              precision    recall  f1-score   support

           0       0.76      0.71      0.74      1024
           1       0.75      0.80      0.77      1116

    accuracy                           0.76      2140
   macro avg       0.76      0.76      0.76      2140
weighted avg       0.76      0.76      0.76      2140



In [40]:
from sklearn.ensemble import GradientBoostingClassifier
clf=Pipeline([
    ('gbc',GradientBoostingClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfgbs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfgbs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfgbs.pkl')

              precision    recall  f1-score   support

           0       0.89      0.79      0.84      1024
           1       0.83      0.91      0.87      1116

    accuracy                           0.86      2140
   macro avg       0.86      0.85      0.85      2140
weighted avg       0.86      0.86      0.85      2140



In [41]:
x_train, x_test, y_train, y_test = train_test_split(df_combined['lemmatized_content'],df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label'])

In [42]:
X_train_vectorized = np.array([get_sentence_embedding(text, glove_dict) for text in x_train])
X_test_vectorized = np.array([get_sentence_embedding(text, glove_dict) for text in x_test])

In [43]:
clf = Pipeline([
    ('rfc', RandomForestClassifier())
])
clf.fit(X_train_vectorized, y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfrfl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfrfl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfrfl.pkl')

              precision    recall  f1-score   support

           0       0.88      0.89      0.89      1020
           1       0.90      0.89      0.89      1120

    accuracy                           0.89      2140
   macro avg       0.89      0.89      0.89      2140
weighted avg       0.89      0.89      0.89      2140



In [44]:
clf=Pipeline([
    ('svc',SVC())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfsvcl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfsvl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfsvl.pkl')

              precision    recall  f1-score   support

           0       0.92      0.87      0.90      1020
           1       0.89      0.93      0.91      1120

    accuracy                           0.91      2140
   macro avg       0.91      0.90      0.91      2140
weighted avg       0.91      0.91      0.91      2140



In [45]:
clf=Pipeline([
    ('lr',LogisticRegression())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclflrl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclflrl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclflrl.pkl')

              precision    recall  f1-score   support

           0       0.89      0.84      0.86      1020
           1       0.86      0.90      0.88      1120

    accuracy                           0.87      2140
   macro avg       0.87      0.87      0.87      2140
weighted avg       0.87      0.87      0.87      2140



In [46]:
clf=Pipeline([
    ('knn',KNeighborsClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfknnl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfknnl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfknnl.pkl')

              precision    recall  f1-score   support

           0       0.93      0.80      0.86      1020
           1       0.84      0.95      0.89      1120

    accuracy                           0.87      2140
   macro avg       0.88      0.87      0.87      2140
weighted avg       0.88      0.87      0.87      2140



In [47]:
clf=Pipeline([
    ('dtc',DecisionTreeClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfdtl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfdtl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfdtl.pkl')

              precision    recall  f1-score   support

           0       0.77      0.76      0.77      1020
           1       0.79      0.79      0.79      1120

    accuracy                           0.78      2140
   macro avg       0.78      0.78      0.78      2140
weighted avg       0.78      0.78      0.78      2140



In [48]:
clf=Pipeline([
    ('gbc',GradientBoostingClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
pclfgbl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'pclfgbl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('pclfgbl.pkl')

              precision    recall  f1-score   support

           0       0.88      0.86      0.87      1020
           1       0.87      0.90      0.89      1120

    accuracy                           0.88      2140
   macro avg       0.88      0.88      0.88      2140
weighted avg       0.88      0.88      0.88      2140



# **Praveen**

In [310]:
df_combined.head()

,Id,content,label,create_time,stemming_content,lemmatized_content
0,1,From @lindseyvonn: The 90 seconds that changed...,0,1.695917e+09,lindseyvonn 90 second chang life http,lindseyvonn 90 second changed life http
1,2,The Supreme Court in Biden v. Nebraska has str...,0,1.688054e+09,suprem court biden nebraska struck presid bide...,Supreme Court Biden Nebraska struck President ...
2,3,U.S. nerve gas hit our own troops in Iraq: htt...,0,1.427731e+09,nerv ga hit troop iraq http http,nerve gas hit troop Iraq http http
3,4,The Lockdown Files is the biggest leak of data...,0,1.677514e+09,lockdown file biggest leak data involv govern ...,Lockdown Files biggest leak data involving Gov...
4,5,What is impeachment and how does it work? @Pet...,0,1.572451e+09,impeach work petewilliamsnbc explain nbcnewsno...,impeachment work PeteWilliamsNBC explains NBCN...


In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import word_tokenize
from gensim.models import KeyedVectors
nltk.download('punkt')

# Load pre-trained Word2Vec model
def load_word2vec_model(word2vec_path):
    return KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

# Convert text data into Word2Vec-based sentence embeddings
def get_sentence_embedding(sentence, word2vec_model, embedding_dim=100):
    words = word_tokenize(sentence.lower())
    word_vectors = [word2vec_model[word] for word in words if word in word2vec_model]
    if len(word_vectors) == 0:
        return np.zeros(embedding_dim)
    return np.mean(word_vectors, axis=0)

x_train, x_test, y_train, y_test = train_test_split(df['tweet'], df['label'], test_size=0.2, random_state=2022)

# Load Word2Vec vectors
word2vec_path = "GoogleNews-vectors-negative300.bin"  
word2vec_model = load_word2vec_model(word2vec_path)

# Convert text data to embeddings
X_train_vectorized = np.array([get_sentence_embedding(text, word2vec_model, embedding_dim=300) for text in x_train])
X_test_vectorized = np.array([get_sentence_embedding(text, word2vec_model, embedding_dim=300) for text in x_test])

# Train Random Forest model
clf = Pipeline([
    ('rf', RandomForestClassifier())
])
clf.fit(X_train_vectorized, y_train)

# Predictions
y_pred = clf.predict(X_test_vectorized)
print(classification_report(y_test, y_pred))
dclfrf = clf.score(X_test_vectorized, y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfrf.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfrf.pkl')



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\91935\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\91935\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\91935\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\91935\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\91935\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "C:\Users\91935\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\91935\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "C:\Users\91935\anaconda3\Lib\site-pack

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



C:\Users\91935\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [58]:
clf=Pipeline([
    ('svc',SVC())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfsvc=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfsvc.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfsvc.pkl')


              precision    recall  f1-score   support

        fake       0.95      0.93      0.94       620
        real       0.94      0.96      0.95       664

    accuracy                           0.95      1284
   macro avg       0.95      0.95      0.95      1284
weighted avg       0.95      0.95      0.95      1284



In [59]:
clf=Pipeline([
    ('lr',LogisticRegression())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclflr=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclflr.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclflr.pkl')

              precision    recall  f1-score   support

        fake       0.92      0.90      0.91       620
        real       0.91      0.93      0.92       664

    accuracy                           0.91      1284
   macro avg       0.91      0.91      0.91      1284
weighted avg       0.91      0.91      0.91      1284



In [60]:
clf=Pipeline([
    ('knn',KNeighborsClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfknn=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfknn.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfknn.pkl')

              precision    recall  f1-score   support

        fake       0.95      0.81      0.88       620
        real       0.85      0.96      0.90       664

    accuracy                           0.89      1284
   macro avg       0.90      0.89      0.89      1284
weighted avg       0.90      0.89      0.89      1284



In [61]:
clf=Pipeline([
    ('dtc',DecisionTreeClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfdt=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfdt.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfdt.pkl')

              precision    recall  f1-score   support

        fake       0.80      0.78      0.79       620
        real       0.80      0.81      0.81       664

    accuracy                           0.80      1284
   macro avg       0.80      0.80      0.80      1284
weighted avg       0.80      0.80      0.80      1284



In [62]:
clf=Pipeline([
    ('gbc',GradientBoostingClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfgb=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfgb.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfgb.pkl')

              precision    recall  f1-score   support

        fake       0.93      0.90      0.92       620
        real       0.91      0.93      0.92       664

    accuracy                           0.92      1284
   macro avg       0.92      0.92      0.92      1284
weighted avg       0.92      0.92      0.92      1284



In [63]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_combined['stemming_content'],df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label'])

In [64]:
X_train_vectorized = np.array([get_sentence_embedding(text, word2vec_model, embedding_dim=300) for text in x_train])
X_test_vectorized = np.array([get_sentence_embedding(text, word2vec_model, embedding_dim=300) for text in x_test])

In [65]:
clf = Pipeline([
    ('rfc', RandomForestClassifier())
])
clf.fit(X_train_vectorized, y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfrfs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfrfs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfrfs.pkl')

              precision    recall  f1-score   support

           0       0.85      0.90      0.87      1020
           1       0.90      0.86      0.88      1120

    accuracy                           0.88      2140
   macro avg       0.88      0.88      0.88      2140
weighted avg       0.88      0.88      0.88      2140



In [66]:
clf=Pipeline([
    ('svc',SVC())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfsvcs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfsvcs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfsvcs.pkl')

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      1020
           1       0.92      0.93      0.92      1120

    accuracy                           0.92      2140
   macro avg       0.92      0.92      0.92      2140
weighted avg       0.92      0.92      0.92      2140



In [76]:
clf=Pipeline([
    ('lr',LogisticRegression())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclflrs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclflrs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclflrs.pkl')

              precision    recall  f1-score   support

           0       0.91      0.88      0.90      1020
           1       0.89      0.92      0.91      1120

    accuracy                           0.90      2140
   macro avg       0.90      0.90      0.90      2140
weighted avg       0.90      0.90      0.90      2140



In [77]:
clf=Pipeline([
    ('knn',KNeighborsClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfknns=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfknns.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfknns.pkl')

              precision    recall  f1-score   support

           0       0.92      0.79      0.85      1020
           1       0.83      0.94      0.88      1120

    accuracy                           0.87      2140
   macro avg       0.88      0.87      0.87      2140
weighted avg       0.88      0.87      0.87      2140



In [78]:
clf=Pipeline([
    ('dtc',DecisionTreeClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfdts=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfdts.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfdts.pkl')

              precision    recall  f1-score   support

           0       0.78      0.78      0.78      1020
           1       0.80      0.80      0.80      1120

    accuracy                           0.79      2140
   macro avg       0.79      0.79      0.79      2140
weighted avg       0.79      0.79      0.79      2140



In [70]:
clf=Pipeline([
    ('gbc',GradientBoostingClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfgbs=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfgbs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfgbs.pkl')

              precision    recall  f1-score   support

           0       0.88      0.86      0.87      1020
           1       0.87      0.89      0.88      1120

    accuracy                           0.88      2140
   macro avg       0.88      0.88      0.88      2140
weighted avg       0.88      0.88      0.88      2140



In [71]:
x_train, x_test, y_train, y_test = train_test_split(df_combined['lemmatized_content'],df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label'])

In [72]:
X_train_vectorized = np.array([get_sentence_embedding(text, word2vec_model, embedding_dim=300) for text in x_train])
X_test_vectorized = np.array([get_sentence_embedding(text, word2vec_model, embedding_dim=300) for text in x_test])

In [74]:
clf = Pipeline([
    ('rfc', RandomForestClassifier())
])
clf.fit(X_train_vectorized, y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfrfl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfrfl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfrfl.pkl')

              precision    recall  f1-score   support

           0       0.88      0.90      0.89      1020
           1       0.91      0.89      0.90      1120

    accuracy                           0.89      2140
   macro avg       0.89      0.89      0.89      2140
weighted avg       0.89      0.89      0.89      2140



In [75]:
clf=Pipeline([
    ('svc',SVC())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfsvcl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfsvcl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfsvcl.pkl')

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      1020
           1       0.93      0.94      0.93      1120

    accuracy                           0.93      2140
   macro avg       0.93      0.93      0.93      2140
weighted avg       0.93      0.93      0.93      2140



In [80]:
clf=Pipeline([
    ('lr',LogisticRegression())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclflrl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclflrl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclflrl.pkl')

              precision    recall  f1-score   support

           0       0.91      0.88      0.90      1020
           1       0.89      0.92      0.91      1120

    accuracy                           0.90      2140
   macro avg       0.90      0.90      0.90      2140
weighted avg       0.90      0.90      0.90      2140



In [81]:
clf=Pipeline([
    ('knn',KNeighborsClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfknnl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfknnl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfknnl.pkl')

              precision    recall  f1-score   support

           0       0.92      0.79      0.85      1020
           1       0.83      0.94      0.88      1120

    accuracy                           0.87      2140
   macro avg       0.88      0.87      0.87      2140
weighted avg       0.88      0.87      0.87      2140



In [82]:
clf=Pipeline([
    ('dtc',DecisionTreeClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfdtl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfdtl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfdtl.pkl')

              precision    recall  f1-score   support

           0       0.78      0.78      0.78      1020
           1       0.80      0.79      0.80      1120

    accuracy                           0.79      2140
   macro avg       0.79      0.79      0.79      2140
weighted avg       0.79      0.79      0.79      2140



In [83]:
clf=Pipeline([
    ('gbc',GradientBoostingClassifier())
])
clf.fit(X_train_vectorized,y_train)
y_pred=clf.predict(X_test_vectorized)
print(classification_report(y_test,y_pred))
dclfgbl=clf.score(X_test_vectorized,y_test)
import joblib

# Save the trained model
joblib.dump(clf, 'dclfgbl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('dclfgbl.pkl')

              precision    recall  f1-score   support

           0       0.90      0.87      0.89      1020
           1       0.89      0.91      0.90      1120

    accuracy                           0.89      2140
   macro avg       0.89      0.89      0.89      2140
weighted avg       0.89      0.89      0.89      2140



# **Ranjith**

In [45]:
import pandas as pd
import fasttext
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier
import nltk
from nltk.tokenize import word_tokenize



# Download NLTK tokenizer
nltk.download('punkt')

# Load datasets
df = pd.read_excel("Constraint_English_Train.xlsx")
df1 = pd.read_excel("english_test_with_labels.xlsx")
df2 = pd.read_excel("Constraint_English_Val.xlsx")

# Combine datasets
df_combined = pd.concat([df, df1, df2])

# Convert labels to binary (1 = "real", 0 = "fake")
df_combined["label"] = df_combined["label"].apply(lambda x: 1 if x == "real" else 0)

# Split dataset
x_train, x_test, y_train, y_test = train_test_split(
    df_combined['tweet'], df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label']
)

# Save training data for FastText
train_text_file = 'fasttext_train.txt'
with open(train_text_file, 'w', encoding='utf-8') as f:
    for tweet, label in zip(x_train, y_train):
        clean_tweet = str(tweet).replace('\n', ' ').strip()
        f.write(f"__label__{label} {clean_tweet}\n")

# Train FastText model
fasttext_model = fasttext.train_supervised(input=train_text_file, epoch=25, lr=0.5, wordNgrams=2)

# Function to get FastText embeddings
def get_sentence_vector(text):
    if isinstance(text, str):
        text = text.replace('\n', ' ').strip()
    else:
        text = ""
    return fasttext_model.get_sentence_vector(text)

# Convert text to FastText embeddings
x_train_embeddings = np.array([get_sentence_vector(text) for text in x_train])
x_test_embeddings = np.array([get_sentence_vector(text) for text in x_test])

# Define and train the classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(x_train_embeddings, y_train)

# Predictions
y_pred = clf.predict(x_test_embeddings)
print(classification_report(y_test, y_pred))

# Model Accuracy
rclfrf = clf.score(x_test_embeddings, y_test)
print("Random Forest Accuracy:", rclfrf)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfrf.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfrf.pkl')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1020
           1       0.96      0.93      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

Random Forest Accuracy: 0.9415887850467289


C:\Users\91935\NLP\new fatstext code\rclfrf.pkl

In [47]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Define & train SVM model
clf_svc = SVC(kernel='linear')
clf_svc.fit(x_train_embeddings, y_train)

# Predictions
y_pred = clf_svc.predict(x_test_embeddings)

# Print Classification Report
print(classification_report(y_test, y_pred))

# Model Accuracy
rclfsvc = clf_svc.score(x_test_embeddings, y_test)
print("SVM Accuracy:", rclfsvc)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfsvc.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfsvc.pkl')


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1020
           1       0.96      0.93      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

SVM Accuracy: 0.9415887850467289


C:\Users\91935\NLP\new fatstext code\rclfsvc.pkl

In [49]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Define and train Logistic Regression model
clf_lr = LogisticRegression()
clf_lr.fit(x_train_embeddings, y_train)

# Predictions
y_pred = clf_lr.predict(x_test_embeddings)

# Print Classification Report
print(classification_report(y_test, y_pred))

# Model Accuracy
rclflr = clf_lr.score(x_test_embeddings, y_test)
print("Logistic Regression Accuracy:", rclflr)
import joblib

# Save the trained model
joblib.dump(clf, 'rclflr.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclflr.pkl')

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1020
           1       0.95      0.94      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

Logistic Regression Accuracy: 0.9401869158878504


C:\Users\91935\NLP\new fatstext code\rclflr.pkl

In [51]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report

# Define and train KNN model
clf_knn = KNeighborsClassifier(n_neighbors=5)
clf_knn.fit(x_train_embeddings, y_train)

# Predictions
y_pred = clf_knn.predict(x_test_embeddings)

# Print Classification Report
print(classification_report(y_test, y_pred))

# Model Accuracy
rclf_knn = clf_knn.score(x_test_embeddings, y_test)
print("KNN Accuracy:", rclf_knn)
import joblib

# Save the trained model
joblib.dump(clf, 'rclf_knn.pkl')
from IPython.display import FileLink

# Provide a download link


              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1020
           1       0.96      0.93      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

KNN Accuracy: 0.9415887850467289


C:\Users\91935\NLP\new fatstext code\rcfl_knn.pkl

In [53]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Define and train Decision Tree model
clf_dt = DecisionTreeClassifier()
clf_dt.fit(x_train_embeddings, y_train)

# Predictions
y_pred = clf_dt.predict(x_test_embeddings)

# Print Classification Report
print(classification_report(y_test, y_pred))

# Model Accuracy
rclf_dt = clf_dt.score(x_test_embeddings, y_test)
print("Decision Tree Accuracy:", rclf_dt)
import joblib

# Save the trained model
joblib.dump(clf, 'rclf_dt.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclf_dt.pkl')

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1020
           1       0.96      0.93      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

Decision Tree Accuracy: 0.9429906542056075


C:\Users\91935\NLP\new fatstext code\rclf_dt.pkl

In [57]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report

# Define and train Gradient Boosting model
clf_gb = GradientBoostingClassifier()
clf_gb.fit(x_train_embeddings, y_train)

# Predictions
y_pred = clf_gb.predict(x_test_embeddings)

# Print Classification Report
print(classification_report(y_test, y_pred))

# Model Accuracy
rclfgb = clf_gb.score(x_test_embeddings, y_test)
print("Gradient Boosting Accuracy:", rclfgb)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfgb.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfgb.pkl')

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      1020
           1       0.95      0.93      0.94      1120

    accuracy                           0.94      2140
   macro avg       0.94      0.94      0.94      2140
weighted avg       0.94      0.94      0.94      2140

Gradient Boosting Accuracy: 0.9411214953271028


C:\Users\91935\NLP\new fatstext code\rclfgb.pkl

In [59]:
import nltk
import numpy as np
import fasttext
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split

# Download necessary nltk resources
nltk.download('punkt')
nltk.download('wordnet')

# Initialize stemmer and lemmatizer
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

# Function for stemming
def stem_text(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        return " ".join([stemmer.stem(word) for word in words])
    return ""

# Function for lemmatization
def lemmatize_text(text):
    if isinstance(text, str):
        words = word_tokenize(text)
        return " ".join([lemmatizer.lemmatize(word) for word in words])
    return ""

# Ensure df_combined exists before running this
if "df_combined" not in locals():
    raise ValueError("df_combined is not defined. Please load your dataset.")

# Apply stemming and lemmatization
df_combined["stemmed_tweet"] = df_combined["tweet"].apply(stem_text)
df_combined["lemmatized_tweet"] = df_combined["tweet"].apply(lemmatize_text)

# Split data for stemming
x_train_stem, x_test_stem, y_train_stem, y_test_stem = train_test_split(
    df_combined['stemmed_tweet'], df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label']
)

# Split data for lemmatization
x_train_lem, x_test_lem, y_train_lem, y_test_lem = train_test_split(
    df_combined['lemmatized_tweet'], df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label']
)

# Save training data for FastText (stemming)
with open("fasttext_train_stem.txt", 'w', encoding='utf-8') as f:
    for tweet, label in zip(x_train_stem, y_train_stem):
        clean_tweet = tweet.replace('\n', ' ').strip()
        f.write(f"__label__{label} {clean_tweet}\n")

# Save training data for FastText (lemmatization)
with open("fasttext_train_lem.txt", 'w', encoding='utf-8') as f:
    for tweet, label in zip(x_train_lem, y_train_lem):
        clean_tweet = tweet.replace('\n', ' ').strip()
        f.write(f"__label__{label} {clean_tweet}\n")

# Train FastText models
fasttext_model_stem = fasttext.train_supervised(input="fasttext_train_stem.txt", epoch=25, lr=0.5, wordNgrams=2)
fasttext_model_lem = fasttext.train_supervised(input="fasttext_train_lem.txt", epoch=25, lr=0.5, wordNgrams=2)

# Function to get FastText embeddings
def get_sentence_vector(text, model):
    if isinstance(text, str):
        text = text.replace('\n', ' ').strip()
    else:
        text = ""
    return model.get_sentence_vector(text)

# Convert text to FastText embeddings (stemming)
x_train_stem_embeddings = np.array([get_sentence_vector(text, fasttext_model_stem) for text in x_train_stem])
x_test_stem_embeddings = np.array([get_sentence_vector(text, fasttext_model_stem) for text in x_test_stem])

# Convert text to FastText embeddings (lemmatization)
x_train_lem_embeddings = np.array([get_sentence_vector(text, fasttext_model_lem) for text in x_train_lem])
x_test_lem_embeddings = np.array([get_sentence_vector(text, fasttext_model_lem) for text in x_test_lem])


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\91935\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [61]:
rclfrfs = RandomForestClassifier(random_state=2022)
rclfrfs.fit(x_train_stem_embeddings, y_train_stem)
y_pred_stem = rclfrfs.predict(x_test_stem_embeddings)
print("Stemming Model Performance:")
print(classification_report(y_test_stem, y_pred_stem))
accuracy_stem = rclfrfs.score(x_test_stem_embeddings, y_test_stem)
print("Random Forest Accuracy (Stemming):", accuracy_stem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfrfs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfrfs.pkl')

Stemming Model Performance:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.96      0.97      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

Random Forest Accuracy (Stemming): 0.9616822429906542


C:\Users\91935\NLP\new fatstext code\rclfrfs.pkl

In [63]:
rclfsvms = SVC(kernel="linear", random_state=2022)

# Train and evaluate SVM on stemming
print("\n==== SVM Performance (Stemming) ====")
rclfsvms.fit(x_train_stem_embeddings, y_train_stem)
y_pred_stem = rclfsvms.predict(x_test_stem_embeddings)
print(classification_report(y_test_stem, y_pred_stem))
accuracy_stem = rclfsvms.score(x_test_stem_embeddings, y_test_stem)
print("SUPPORT VECTOR MACHINE (Stemming):", accuracy_stem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfsvms.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfsvms.pkl')


==== SVM Performance (Stemming) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.96      0.96      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

SUPPORT VECTOR MACHINE (Stemming): 0.9602803738317757


C:\Users\91935\NLP\new fatstext code\rclfsvms.pkl

In [65]:
rclflrs = LogisticRegression(max_iter=500, random_state=2022)
print("\n==== Logistic Regression Performance (Stemming) ====")
rclflrs.fit(x_train_stem_embeddings, y_train_stem)
y_pred_stem = rclflrs.predict(x_test_stem_embeddings)
print(classification_report(y_test_stem, y_pred_stem))
accuracy_stem = rclflrs.score(x_test_stem_embeddings, y_test_stem)
print("logistic regression (Stemming):", accuracy_stem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclflrs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclflrs.pkl')


==== Logistic Regression Performance (Stemming) ====
              precision    recall  f1-score   support

           0       0.97      0.95      0.96      1020
           1       0.96      0.97      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

logistic regression (Stemming): 0.9616822429906542


C:\Users\91935\NLP\new fatstext code\rclflrs.pkl

In [67]:
rclfknns = KNeighborsClassifier(n_neighbors=5)
print("\n==== KNN Performance (Stemming) ====")
rclfknns.fit(x_train_stem_embeddings, y_train_stem)
y_pred_stem = rclfknns.predict(x_test_stem_embeddings)
print(classification_report(y_test_stem, y_pred_stem))
accuracy_stem = rclfknns.score(x_test_stem_embeddings, y_test_stem)
print("logistic regression (Stemming):", accuracy_stem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfknns.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfknns.pkl')


==== KNN Performance (Stemming) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.96      0.96      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

logistic regression (Stemming): 0.9607476635514018


C:\Users\91935\NLP\new fatstext code\rclfknns.pkl

In [69]:
rclfdts = DecisionTreeClassifier(random_state=2022)
print("\n==== Decision Tree Performance (Stemming) ====")
rclfdts.fit(x_train_stem_embeddings, y_train_stem)
y_pred_stem = rclfdts.predict(x_test_stem_embeddings)
print(classification_report(y_test_stem, y_pred_stem))
accuracy_stem = rclfdts.score(x_test_stem_embeddings, y_test_stem)
print("decission tree (Stemming):", accuracy_stem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfdts.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfdts.pkl')


==== Decision Tree Performance (Stemming) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.96      0.96      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

decission tree (Stemming): 0.9598130841121495


C:\Users\91935\NLP\new fatstext code\rclfdts.pkl

In [71]:
rclfgbs = GradientBoostingClassifier(random_state=2022)
print("\n==== Gradient Boosting Performance (Stemming) ====")
rclfgbs.fit(x_train_stem_embeddings, y_train_stem)
y_pred_stem = rclfgbs.predict(x_test_stem_embeddings)
print(classification_report(y_test_stem, y_pred_stem))
accuracy_stem = rclfgbs.score(x_test_stem_embeddings, y_test_stem)
print("gradient boosting (Stemming):", accuracy_stem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfgbs.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfgbs.pkl')


==== Gradient Boosting Performance (Stemming) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.96      0.97      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

gradient boosting (Stemming): 0.961214953271028


C:\Users\91935\NLP\new fatstext code\rclfgbs.pkl

#df_combined["lemmatized_tweet"] = df_combined["tweet"].apply(lemmatize_text)
#x_train_lem, x_test_lem, y_train_lem, y_test_lem = train_test_split(
 #   df_combined['lemmatized_tweet'], df_combined['label'], test_size=0.2, random_state=2022, stratify=df_combined['label']
#)not usefull lemmatizaton is predefined

In [73]:
rclfrfl = RandomForestClassifier(random_state=2022)
rclfrfl.fit(x_train_lem_embeddings, y_train_lem)
y_pred_lem = rclfrfl.predict(x_test_lem_embeddings)
print("Lemmatization Model Performance:")
print(classification_report(y_test_lem, y_pred_lem))
accuracy_lem = rclfrfl.score(x_test_lem_embeddings, y_test_lem)
print("Random Forest Accuracy (Lemmatization):", accuracy_lem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfrfl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfrfl.pkl')

Lemmatization Model Performance:
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.97      0.97      0.97      1120

    accuracy                           0.97      2140
   macro avg       0.97      0.97      0.97      2140
weighted avg       0.97      0.97      0.97      2140

Random Forest Accuracy (Lemmatization): 0.9654205607476636


C:\Users\91935\NLP\new fatstext code\rclfrfl.pkl

In [75]:
rclfsvml= SVC(kernel="linear", random_state=2022)
print("\n==== SVM Performance (Lemmatization) ====")
rclfsvml.fit(x_train_lem_embeddings, y_train_lem)
y_pred_lem =rclfsvml.predict(x_test_lem_embeddings)
print(classification_report(y_test_lem, y_pred_lem))
accuracy_lem = rclfsvml.score(x_test_lem_embeddings, y_test_lem)
print("SVM (Lemmatization):", accuracy_lem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfsvml.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfsvml.pkl')


==== SVM Performance (Lemmatization) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.97      0.97      0.97      1120

    accuracy                           0.97      2140
   macro avg       0.97      0.97      0.97      2140
weighted avg       0.97      0.97      0.97      2140

SVM (Lemmatization): 0.9654205607476636


C:\Users\91935\NLP\new fatstext code\rclfsvml.pkl

In [77]:
rclflrl = LogisticRegression(max_iter=500, random_state=2022)
print("\n==== Logistic Regression Performance (Lemmatization) ====")
rclflrl.fit(x_train_lem_embeddings, y_train_lem)
y_pred_lem = rclflrl.predict(x_test_lem_embeddings)
print(classification_report(y_test_lem, y_pred_lem))
accuracy_lem = rclflrl.score(x_test_lem_embeddings, y_test_lem)
print("LogisticRegression (Lemmatization):", accuracy_lem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclflrl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclflrl.pkl')


==== Logistic Regression Performance (Lemmatization) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.96      0.97      0.96      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

LogisticRegression (Lemmatization): 0.9630841121495327


C:\Users\91935\NLP\new fatstext code\rclflrl.pkl

In [79]:
rclfcnnl= KNeighborsClassifier(n_neighbors=5)
print("\n==== KNN Performance (Lemmatization) ====")
rclfcnnl.fit(x_train_lem_embeddings, y_train_lem)
y_pred_lem = rclfcnnl.predict(x_test_lem_embeddings)
print(classification_report(y_test_lem, y_pred_lem))
accuracy_lem = rclfcnnl.score(x_test_lem_embeddings, y_test_lem)
print(" KNeighborsClassifie(Lemmatization):", accuracy_lem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfcnnl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfcnnl.pkl')


==== KNN Performance (Lemmatization) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.97      0.97      0.97      1120

    accuracy                           0.97      2140
   macro avg       0.97      0.97      0.97      2140
weighted avg       0.97      0.97      0.97      2140

 KNeighborsClassifie(Lemmatization): 0.9654205607476636


C:\Users\91935\NLP\new fatstext code\rclfcnnl.pkl

In [81]:
from sklearn.tree import DecisionTreeClassifier
rclfdtl= DecisionTreeClassifier(random_state=42)
print("\n==== Decision Tree Performance (Lemmatization) ====")
rclfdtl.fit(x_train_lem_embeddings, y_train_lem)
y_pred_lem = rclfdtl.predict(x_test_lem_embeddings)
print(classification_report(y_test_lem, y_pred_lem))
accuracy_lem = rclfdtl.score(x_test_lem_embeddings, y_test_lem)
print("DECISSION TREE (Lemmatization):", accuracy_lem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfdtl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfdtl.pkl')


==== Decision Tree Performance (Lemmatization) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.97      0.97      0.97      1120

    accuracy                           0.96      2140
   macro avg       0.96      0.96      0.96      2140
weighted avg       0.96      0.96      0.96      2140

DECISSION TREE (Lemmatization): 0.9644859813084112


C:\Users\91935\NLP\new fatstext code\rclfdtl.pkl

In [83]:
rclfgbl = GradientBoostingClassifier(random_state=2022)
print("\n==== Gradient Boosting Performance (Lemmatization) ====")
rclfgbl.fit(x_train_lem_embeddings, y_train_lem)
y_pred_lem = rclfgbl.predict(x_test_lem_embeddings)
print(classification_report(y_test_lem, y_pred_lem))
accuracy_lem = rclfgbl.score(x_test_lem_embeddings, y_test_lem)
print("GRADIENT BOOSTING", accuracy_lem)
import joblib

# Save the trained model
joblib.dump(clf, 'rclfgbl.pkl')
from IPython.display import FileLink

# Provide a download link
FileLink('rclfgbl.pkl')


==== Gradient Boosting Performance (Lemmatization) ====
              precision    recall  f1-score   support

           0       0.96      0.96      0.96      1020
           1       0.97      0.97      0.97      1120

    accuracy                           0.97      2140
   macro avg       0.97      0.97      0.97      2140
weighted avg       0.97      0.97      0.97      2140

GRADIENT BOOSTING 0.9654205607476636


C:\Users\91935\NLP\new fatstext code\rclfgbl.pkl